In [ ]:
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import datetime

# 第一步 加载模型

In [ ]:
# 指定使用的预训练模型名称
model_name = "path/to/your/model"  # 替换为你自己的模型路径或名称

In [ ]:
# 从预训练模型中加载一个用于序列分类的模型
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 第二步 加载分词器

In [ ]:
# 加载对应的分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 第三步 加载数据集

In [ ]:
# 加载你的数据集
dataset = load_dataset("your_dataset_name")  # 替换为你自己的数据集名称

# 第四步 定义预处理函数

In [ ]:
# 定义一个预处理函数，用于对数据集中的句子对进行分词和截断
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=True)


In [ ]:
# 对数据集进行预处理，将句子对转换为模型可以接受的输入格式
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# 使用 DataCollatorWithPadding 进行动态填充
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 第五步 定义训练参数

In [ ]:
# 定义训练参数，包括输出目录等
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",  # 使用新的参数名 eval_strategy
    logging_dir="logs",
    logging_steps=10,
)

# 第六步  创建训练

In [ ]:
# 创建一个 Trainer 实例，用于训练和评估模型
trainer = Trainer(
    model=model,  # 模型
    args=training_args,  # 训练参数
    train_dataset=tokenized_datasets["train"],  # 训练数据集
    eval_dataset=tokenized_datasets["validation"],  # 验证数据集
    data_collator=data_collator,  # 动态填充
)

# 第七步 训练模型&评估

In [ ]:
# 进行评估
eval_result = trainer.evaluate()

# 第八步 打印结果&保存指标

In [ ]:
# 打印评估结果
print(eval_result)


In [ ]:
# 提取指标名称和对应的值
metrics = list(eval_result.keys())
values = list(eval_result.values())

In [ ]:
# 创建图表
plt.figure(figsize=(10, 6))
plt.bar(metrics, values, color='skyblue')
plt.xlabel('Metrics指标')
plt.ylabel('Values')
plt.title('Evaluation Metrics评估指标')
plt.ylim(0, 1)  # 设置 y 轴的范围
for i, value in enumerate(values):
    plt.text(i, value + 0.02, f'{value:.2f}', ha='center')

In [ ]:
# 获取当前日期和时间
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# 生成文件名
filename = f"{model_name}_{current_time}.png"

# 保存图表
plt.savefig(filename, bbox_inches='tight')

In [ ]:
# 显示图表
plt.show()

print(f"图表已保存为 {filename}")

# 其它说明
模型和分词器：将 model_name 替换为你自己的模型路径或名称。如果你的模型在本地，可以使用路径，如果在 Hugging Face Model Hub 上，可以使用模型名称。
数据集：将 load_dataset("your_dataset_name") 替换为你自己的数据集。如果是本地数据集，确保使用正确的加载方法。
评估结果：代码会自动对模型进行评估，并生成评估结果的图表。
# 自定义部分
如果你的模型和数据集有特定的预处理需求，请调整 preprocess_function。
确保你的模型和数据集兼容 Hugging Face 的 Trainer API。